In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import math as mt

In [ ]:
df = pd.read_csv('mw.csv')
df['GENDER'] -=1
df.head()

,GENDER,HEIGHT,HAND_LENGTH,FOOT_LENGTH
0,0,1760.2,208.6,269.6
1,0,1730.1,207.6,251.3
2,0,1659.6,173.2,193.6
3,0,1751.3,258.0,223.8
4,0,1780.6,212.3,282.1


In [ ]:
df.tail()

,GENDER,HEIGHT,HAND_LENGTH,FOOT_LENGTH
150,1,1541.3,182.6,225.7
151,1,1604.8,188.0,230.6
152,1,1592.1,188.9,232.0
153,1,1518.3,184.6,227.2
154,1,1553.6,186.8,230.3


In [ ]:
male_df = df[df['GENDER']==0]
female_df = df[df['GENDER']==1]
male_df.shape

(80, 4)

In [ ]:
female_df.shape

(75, 4)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['GENDER']), df['GENDER'],test_size=0.3,random_state= 42)

X_train.shape,X_test.shape

((108, 3), (47, 3))

In [ ]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

X_train

array([[1713.1,  202.3,  260.4],
       [1790.3,  212.6,  269.3],
       [1534.7,  182.1,  220.6],
       [1712.5,  204.7,  261.5],
       [1708.1,  204.6,  260. ],
       [1630.2,  196. ,  247.2],
       [1732.9,  206.1,  262.5],
       [1780.6,  212.3,  282.1],
       [1769.8,  210.6,  265.2],
       [1617.4,  193.7,  240.3],
       [1562.3,  190.6,  237.9],
       [1888.7,  227.7,  276.4],
       [1737.9,  205.5,  259.3],
       [1572. ,  188.2,  230.5],
       [1804.7,  213.9,  273.9],
       [1685.5,  202.1,  250.9],
       [1582.8,  189.1,  233.4],
       [1583.5,  189.9,  238.2],
       [1760.2,  208.6,  269.6],
       [1611.8,  190.8,  235.3],
       [1858.6,  220.3,  274.8],
       [1694.6,  202.2,  256.1],
       [1597.2,  189.4,  232.4],
       [1759.2,  209. ,  264.1],
       [1611.5,  195.1,  245.3],
       [1619.5,  193.3,  239.9],
       [1771. ,  210.4,  265.7],
       [1748.4,  207.9,  262. ],
       [1682.6,  200.7,  249.7],
       [1689.8,  203.1,  261.9],
       [16

In [ ]:
X_train = np.log(X_train + 1)
X_test = np.log(X_test + 1)

overall_max_train = X_train.max()
overall_max_train

overall_max_test= X_test.max()
overall_max_test

X_train /= overall_max_train
X_test /= overall_max_test

X_test

array([[0.97896744, 0.68979849, 0.71123359],
       [0.97875789, 0.69600206, 0.72329985],
       [0.99319446, 0.71091901, 0.74075195],
       [0.99369766, 0.71079297, 0.74110373],
       [0.97679738, 0.69600206, 0.7235292 ],
       [0.9920837 , 0.71016096, 0.74029829],
       [0.98139076, 0.69865352, 0.72669913],
       [0.97924353, 0.69670496, 0.7243288 ],
       [0.97324283, 0.69394237, 0.7213925 ],
       [0.99250794, 0.70971676, 0.74070162],
       [0.98442337, 0.70492777, 0.73928456],
       [0.98242295, 0.69865352, 0.72534984],
       [0.99314178, 0.71129642, 0.74190432],
       [0.98696313, 0.7074732 , 0.74080226],
       [0.98771804, 0.70643481, 0.73676775],
       [0.99946993, 0.71622868, 0.74956142],
       [0.99693376, 0.71254676, 0.7428488 ],
       [0.98142366, 0.69879162, 0.72681095],
       [0.97402747, 0.68957677, 0.71665167],
       [0.98080559, 0.69789142, 0.72523677],
       [0.99454204, 0.71242225, 0.74309624],
       [0.99021925, 0.70760243, 0.73800644],
       [0.

In [ ]:
np.mean(X_train, axis=0)

array([0.98398975, 0.70220015, 0.73162983])

In [ ]:
class MyDataset(Dataset):
  def __init__(self, x,y):
    self.x = torch.Tensor(x)
    self.y = torch.Tensor(y)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
class BinaryClassifier(nn.Module):
  def __init__(self,input_dim):
      super().__init__()
      self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1),
            nn.Sigmoid()
      )

  def forward(self, x):
    return self.net(x)


In [ ]:
input_dim = X_train.shape[1]

model = BinaryClassifier(input_dim=input_dim)

Lossfunction = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [ ]:
epochs = 50
model.train()
for epoch in range(epochs):
  loss = 0
  running_loss = 0
  correct = 0
  total = 0
  for batch_x,batch_y in train_loader:
    optimizer.zero_grad()
    pred = model(batch_x).squeeze()
    loss = Lossfunction(pred, batch_y)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * batch_x.size(0)
    predicted_labels = (pred >= 0.5).float()
    correct += (predicted_labels == batch_y).sum().item()
    total   += batch_y.size(0)

  avg_loss = running_loss / len(train_dataset)
  accuracy = correct / total
  print(f"Epoch {epoch:2d}/{epochs} — Train Loss: {avg_loss:.4f} — Train accuracy: {accuracy:.4f}")



Epoch  0/50 — Train Loss: 0.6988 — Train accuracy: 0.5093
Epoch  1/50 — Train Loss: 0.6986 — Train accuracy: 0.5093
Epoch  2/50 — Train Loss: 0.6983 — Train accuracy: 0.5093
Epoch  3/50 — Train Loss: 0.6981 — Train accuracy: 0.5093
Epoch  4/50 — Train Loss: 0.6979 — Train accuracy: 0.5093
Epoch  5/50 — Train Loss: 0.6977 — Train accuracy: 0.5093
Epoch  6/50 — Train Loss: 0.6975 — Train accuracy: 0.5093
Epoch  7/50 — Train Loss: 0.6973 — Train accuracy: 0.5093
Epoch  8/50 — Train Loss: 0.6972 — Train accuracy: 0.5093
Epoch  9/50 — Train Loss: 0.6970 — Train accuracy: 0.5093
Epoch 10/50 — Train Loss: 0.6968 — Train accuracy: 0.5093
Epoch 11/50 — Train Loss: 0.6966 — Train accuracy: 0.5093
Epoch 12/50 — Train Loss: 0.6965 — Train accuracy: 0.5093
Epoch 13/50 — Train Loss: 0.6963 — Train accuracy: 0.5093
Epoch 14/50 — Train Loss: 0.6962 — Train accuracy: 0.5093
Epoch 15/50 — Train Loss: 0.6960 — Train accuracy: 0.5093
Epoch 16/50 — Train Loss: 0.6959 — Train accuracy: 0.5093
Epoch 17/50 — 

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for batch_x,batch_y in test_loader:
    pred = model(batch_x).squeeze()
    predicted_labels = (pred >= 0.5).float()
    correct += (predicted_labels == batch_y).sum().item()
    total   += batch_y.size(0)

  accuracy = correct / total
  print(f'Accuracy: {accuracy}')

Accuracy: 0.5319148936170213
